#4-2 ロジスティック回帰

## 概要

**ロジスティック回帰は、いくつかの説明変数から、ある事象の発生する確率を求める手法です。**

この確率を利用して、ある事象が起こる／起こらないという二値分類ができます。
名前に「回帰」とついていますが、回帰問題ではなく、分類問題に使う手法であることに注意してください。
ロジスティック回帰では、シグモイド関数(ロジスティック関数)を出力に使うことにより、出力値が0 ～１の間に収まるように正規化します。そのため、出力値を事象の発生する確率として使用することができます。

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_02_01.jpg?raw=true" width="480px">

また、説明変数の重みの算出には、尤度関数を用いています。
例えば、以下のような分類問題に使用されます。


*   飲酒量、喫煙量、身長、体重から、健康か、不健康かを予測する（二値分類）
*   身長、体重から服のサイズを予測する（多項分類）

## アヤメデータを用いた実装

iris データセットを使ってロジスティック回帰を実際に試してみよう。iris（あやめ）は、セトサ（Iris-Setosa）、バージカラー（Iris-Versicolor）、バージニカ（Iris-Virginica）の3 種類のあやめの
がく片（sepal）と花弁（petal）の幅と長さが収められた有名なデータセットであります。

<img src="https://raw.githubusercontent.com/t-date/DataScience/master/fig/04_06_03.jpg?raw=true" width="460px">

環境設定を行います

In [0]:
# Python 2, 3 をサポートします
from __future__ import division, print_function, unicode_literals

# 標準ライブラリのインポート
import numpy as np
import os

# 乱数の固定
np.random.seed(42)

# プロットの設定
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# 保存先ディレクトリの設定
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "logistic"
IMAGE_DIR = "images"

os.makedirs(os.path.join(PROJECT_ROOT_DIR, IMAGE_DIR, CHAPTER_ID), exist_ok=True)

def image_path(fig_id):
    return os.path.join(PROJECT_ROOT_DIR, IMAGE_DIR , CHAPTER_ID, fig_id)

def save_fig(fig_id, tight_layout=True):
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(image_path(fig_id) + ".png", format='png', dpi=300)

(参考)ロジック関数の描画

In [0]:
t = np.linspace(-10, 10, 100)
sig = 1 / (1 + np.exp(-t))
plt.figure(figsize=(9, 3))
plt.plot([-10, 10], [0, 0], "k-")
plt.plot([-10, 10], [0.5, 0.5], "k:")
plt.plot([-10, 10], [1, 1], "k:")
plt.plot([0, 0], [-1.1, 1.1], "k-")
plt.plot(t, sig, "b-", linewidth=2, label=r"$\sigma(t) = \frac{1}{1 + e^{-t}}$")
plt.xlabel("t")
plt.legend(loc="upper left", fontsize=20)
plt.axis([-10, 10, -0.1, 1.1])
save_fig("logistic_function_plot")
plt.show()

では、花弁の幅特徴量だけでバージニカ種を検出する分類器を作ってみましょう。

In [0]:
from sklearn import datasets
iris = datasets.load_iris()
list(iris.keys())

In [0]:
print(iris.DESCR)

In [0]:
import numpy as np
X = iris["data"][:, 3:] # 花弁の幅
y = (iris["target"] == 2).astype(np.int) # バージニカなら1、他は0

ロジスティック回帰モデルを訓練する。

In [0]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver="liblinear", random_state=42)
log_reg.fit(X, y)

花弁の幅が0cm から3cm の花に対するモデルの推定確率を見てみましょう。

In [0]:
import matplotlib.pyplot as plt
X_new = np.linspace(0, 3, 1000).reshape(-1, 1)
y_proba = log_reg.predict_proba(X_new)
plt.plot(X_new, y_proba[:, 1], "g-", linewidth=2, label="Iris-Virginica")
plt.plot(X_new, y_proba[:, 0], "b--", linewidth=2, label="Not Iris-Virginica")

In [0]:
X_new = np.linspace(0, 3, 1000).reshape(-1, 1)
y_proba = log_reg.predict_proba(X_new)
decision_boundary = X_new[y_proba[:, 1] >= 0.5][0]

plt.figure(figsize=(8, 3))
plt.plot(X[y==0], y[y==0], "bs")
plt.plot(X[y==1], y[y==1], "g^")
plt.plot([decision_boundary, decision_boundary], [-1, 2], "k:", linewidth=2)
plt.plot(X_new, y_proba[:, 1], "g-", linewidth=2, label="Iris-Virginica")
plt.plot(X_new, y_proba[:, 0], "b--", linewidth=2, label="Not Iris-Virginica")
plt.text(decision_boundary+0.02, 0.15, "Decision  boundary", fontsize=14, color="k", ha="center")
plt.arrow(decision_boundary, 0.08, -0.3, 0, head_width=0.05, head_length=0.1, fc='b', ec='b')
plt.arrow(decision_boundary, 0.92, 0.3, 0, head_width=0.05, head_length=0.1, fc='g', ec='g')
plt.xlabel("Petal width (cm)", fontsize=14)
plt.ylabel("Probability", fontsize=14)
plt.legend(loc="center left", fontsize=14)
plt.axis([0, 3, -0.02, 1.02])
save_fig("logistic_regression_plot")
plt.show()

バージニカ種の花弁の幅（三角形）は1.4cm から2.5cm だが、ほかのあやめ（正方形）は0.1cmから1.8cm までの間であります。わずかながら重なり合う部分があることに注意しましょう。

2cm よりも長ければ、分類器はかなり自信を持って、バージニカ種だと判断します（バージニカ種クラスに対して高い確率を出力する）。それに対し、1cm 未満なら、かなり自信を持って、バージニカ種ではないと判断します（非バージニカ種クラスに対して高い確率を出力する）。この両極端の間では、分
類器は自信がなくなる。しかし、クラスの予測を求めればpredict_proba() メソッドではなく、predict() メソッドを使う）、分類器はどちらかのクラスを返します。

そのため、両方の確率が
ともに50% になる決定境界（decision boundary）は、約1.6cm になります。分類器は、花弁の幅が1.6cm よりも長ければバージニカ種、そうでなければ非バージニカ種に分類します。

In [0]:
decision_boundary

In [0]:
log_reg.predict([[1.7], [1.5]])

In [0]:
from sklearn.linear_model import LogisticRegression

X = iris["data"][:, (2, 3)]  # 花弁の長さ、花弁の幅
y = (iris["target"] == 2).astype(np.int)

log_reg = LogisticRegression(solver="liblinear", C=10**10, random_state=42)
log_reg.fit(X, y)

x0, x1 = np.meshgrid(
        np.linspace(2.9, 7, 500).reshape(-1, 1),
        np.linspace(0.8, 2.7, 200).reshape(-1, 1),
    )
X_new = np.c_[x0.ravel(), x1.ravel()]

y_proba = log_reg.predict_proba(X_new)

plt.figure(figsize=(10, 4))
plt.plot(X[y==0, 0], X[y==0, 1], "bs")
plt.plot(X[y==1, 0], X[y==1, 1], "g^")

zz = y_proba[:, 1].reshape(x0.shape)
contour = plt.contour(x0, x1, zz, cmap=plt.cm.brg)


left_right = np.array([2.9, 7])
boundary = -(log_reg.coef_[0][0] * left_right + log_reg.intercept_[0]) / log_reg.coef_[0][1]

plt.clabel(contour, inline=1, fontsize=12)
plt.plot(left_right, boundary, "k--", linewidth=3)
plt.text(3.5, 1.5, "Not Iris-Virginica", fontsize=14, color="b", ha="center")
plt.text(6.5, 2.3, "Iris-Virginica", fontsize=14, color="g", ha="center")
plt.xlabel("Petal length", fontsize=14)
plt.ylabel("Petal width", fontsize=14)
plt.axis([2.9, 7, 0.8, 2.7])
save_fig("logistic_regression_contour_plot")
plt.show()

上図は、同じデータセットを使っているが、今度は花弁の幅と長さのふたつの特徴量を表示しています。訓練すれば、ロジスティック回帰分類器は、これらふたつの特徴量に基づいて、花がバージニカ種である確率を推計できるようになります。

破線は、モデルが50% の確率を推計するところを表しています。これがモデルの決定境界である。この境界が線形になっていることに注意する必要があります。個々の平行線は、モデルが15%（左下）から90%（右上）までの特定の確率を出力する点を表している。モデルによれば、右上の直線の向こう側の花は、90% の確率でバージニカであります。